In [1]:
!pip install pyomo
!pyomo build-extensions

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

**** Building AMPL External function demo library ****
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PkgConfig: /bin/pkg-config (found version "0.29.2") 
-- Configuring done
-- Generating done
-- Build files have been written to: /tmp/tmpfrpd1xbr
[ 50%] Building C object CMakeFiles/asl_external_demo.dir/functions.c.o
[100%] Linking C shared libra

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [10]:
class Data:
  def __init__(self, filename):
    with open(filename) as file_object:
      self.nbFacilities, self.nbRegions, self.nbBeds = [int(n) for n in file_object.readline().split()]
      self.weightCost, self.weightDistance = [float(n) for n in file_object.readline().split()]
      self.isBuilt = []
      self.lbBeds = []
      self.ubBeds = []
      self.costs = []
      for i in range(self.nbFacilities):
        built, lb, ub, cost = file_object.readline().split()
        self.isBuilt.append(bool(built))
        self.lbBeds.append(int(lb))
        self.ubBeds.append(int(ub))
        self.costs.append(0 if bool(built) else int(cost))
      self.demands = [int(n) for n in file_object.readline().split()]
      self.distances = []
      for i in range(self.nbFacilities):
        self.distances.append([int(n) for n in file_object.readline().split()])

In [9]:
data = Data('../instances/df.txt')

In [5]:
class Model:
  def __init__(self, data):
    model = pyo.ConcreteModel()

    model.facilities = range(data.nbFacilities)
    model.regions = range(data.nbRegions)

    model.x = pyo.Var(model.facilities, within=pyo.NonNegativeIntegers)
    model.y = pyo.Var(model.facilities, model.regions, within=pyo.NonNegativeIntegers)
    model.z = pyo.Var(model.facilities, within=pyo.Binary)

    # Objective function
    obj_sum = 0
    for i in model.facilities:
      obj_sum += data.weightCost*data.costs[i]*model.z[i]
      for j in model.regions:
        obj_sum += data.weightDistance*data.distances[i][j]*model.y[i,j]
    model.objective = pyo.Objective(rule=obj_sum, sense=pyo.minimize)

    # Constraints

    model.nb_beds_cons = pyo.Constraint(rule=sum(model.x[i] for i in model.facilities) == data.nbBeds)

    model.capacity_cons = pyo.ConstraintList()
    for i in model.facilities:
      model.capacity_cons.add(sum(model.y[i,j] for j in model.regions) <= model.x[i])

    model.demand_cons = pyo.ConstraintList()
    for j in model.regions:
      model.demand_cons.add(sum(model.y[i,j] for i in model.facilities) == data.demands[j])

    model.x_limits = pyo.ConstraintList()
    for i in model.facilities:
      model.x_limits.add(model.x[i] >= data.lbBeds[i]*model.z[i])
      model.x_limits.add(model.x[i] <= data.ubBeds[i])

    model.z_fix = pyo.ConstraintList()
    for i in model.facilities:
      if data.isBuilt[i]:
        model.z_fix.add(model.z[i] == 1)
    
    model.z_def = pyo.ConstraintList()
    for i in model.facilities:
      model.z_def.add(model.z[i] <= model.x[i])
      model.z_def.add(model.z[i] >= model.x[i]/data.ubBeds[i])

    opt = pyo.SolverFactory('appsi_highs')

    results = opt.solve(model, tee=True)

    results.write()

In [6]:
real_model = Model(Data('../instances/df-real.txt'))

Running HiGHS 1.5.3: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
49 rows, 560 cols, 1120 nonzeros
48 rows, 560 cols, 1120 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   48 rows
   560 cols (80 binary, 480 integer, 0 implied int., 0 continuous)
   1120 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 T       0       0         0   0.00%   0               3934838          100.00%        0      0      0        96     0.0s

Solving report
  Status            Optimal
  Primal bound      3934838
  Dual bound        3934838
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    39

In [7]:
opt_model = Model(Data('../instances/df.txt'))

Running HiGHS 1.5.3: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
49 rows, 573 cols, 1146 nonzeros
49 rows, 573 cols, 1146 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   49 rows
   573 cols (81 binary, 492 integer, 0 implied int., 0 continuous)
   1146 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 T       0       0         0   0.00%   0               3613479          100.00%        0      0      0        81     0.0s

Solving report
  Status            Optimal
  Primal bound      3613479
  Dual bound        3613479
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    36